In [10]:
import sys
sys.path.insert(1, '../../eispy2d/library/')
import pickle
import numpy as np
from numpy import pi
from scipy.constants import epsilon_0, mu_0
from scipy.special import jv, jvp, hankel2, h2vp
import forward as fwr
import inputdata as ipt
import configuration as cfg
import error
from matplotlib import pyplot as plt
import time

In [7]:
name = 'analytical'
NM = NS = 20
Ro = 4.
lambda_b = 1.
epsilon_rb = 1.
Lx = Ly = 2.
E0 = 1.
perfect_dielectric = True

configuration = cfg.Configuration(name=name + '.cfg', number_measurements=NM, 
                           number_sources=NS, observation_radius=Ro,
                           wavelength=lambda_b,
                           background_permittivity=epsilon_rb,
                           image_size=[Ly, Lx], magnitude=E0,
                           perfect_dielectric=perfect_dielectric)

epsilon_rd = 1.5
resolution = (160, 160)
noise = 0.

contrast = (epsilon_rd-epsilon_rb)/epsilon_rb
radius = 0.5
position = [.3, .3]

In [21]:
# Main constants
omega = 2*pi*configuration.f  # Angular frequency [rad/s]
epsilon_rd = cfg.get_relative_permittivity(
    contrast, configuration.epsilon_rb
)
epsd = epsilon_rd*epsilon_0  # Cylinder's permittivity [F/m]
epsb = configuration.epsilon_rb*epsilon_0
mud = mu_0  # Cylinder's permeability [H/m]
kb = configuration.kb  # Wavenumber of background [rad/m]
kd = omega*np.sqrt(mud*epsd)  # Wavenumber of cylinder [rad/m]
lambdab = 2*pi/kb  # Wavelength of background [m]
a = radius*lambdab  # Sphere's radius [m]
thetal = cfg.get_angles(configuration.NS)
thetam = cfg.get_angles(configuration.NM)

# Summing coefficients
tic = time.time()
an, cn, n = get_coefficients(kb, kd, a, epsd, epsb)
toc = time.time()
print('Time to compute coefficients:', toc-tic)

# Mesh parameters
x, y = cfg.get_coordinates_ddomain(
    configuration=configuration,
    resolution=resolution
)
        
# Translate the coordinates to the center of the cylinder
x = x - position[1]
y = y - position[0]

# # Total field array
# tic = time.time()
# et = compute_total_field(x, y, a, an, cn, n, kb, kd,
#                          configuration.E0, thetal)
# toc = time.time()
# print('Time to compute total field:', toc-tic) 

# Scatered field
rho = configuration.Ro
xm = rho*np.cos(thetam) - position[1]
ym = rho*np.sin(thetam) - position[0]
es = compute_scattered_field(xm, ym, an, n, kb, thetal,
                             configuration.E0)



Time to compute coefficients: 0.0029897689819335938


In [19]:
def get_coefficients(wavenumber_b, wavenumber_d, radius, epsilon_d,
                     epsilon_b):
    """Summarize the method."""
    n = np.arange(-1000, 1001)
    kb, kd = wavenumber_b, wavenumber_d
    a = radius

    criteria = np.abs(jvp(n, kd*a))
    n = n[criteria > 1e-25]

    an = (-jv(n, kb*a)/hankel2(n, kb*a)*(
        (epsilon_d*jvp(n, kd*a)/(epsilon_b*kd*a*jv(n, kd*a))
         - jvp(n, kb*a)/(kb*a*jv(n, kb*a)))
        / (epsilon_d*jvp(n, kd*a)/(epsilon_b*kd*a*jv(n, kd*a))
           - h2vp(n, kb*a)/(kb*a*hankel2(n, kb*a)))
    ))

    cn = 1/jv(n, kd*a)*(jv(n, kb*a)+an*hankel2(n, kb*a))

    return an, cn, n

def compute_total_field(x, y, radius, an, cn, N, wavenumber_b, wavenumber_d,
                        magnitude, theta=None):
    """Summarize the method."""
    E0 = magnitude
    kb, kd = wavenumber_b, wavenumber_d
    a = radius

    if theta is None:
        rho, phi = cart2pol(x, y)
        et = np.zeros(rho.shape, dtype=complex)
        i = 0
        for n in N:

            et[rho > a] = et[rho > a] + (
                E0*1j**(-n)*(jv(n, kb*rho[rho > a])
                             + an[i]*hankel2(n, kb*rho[rho > a]))
                * np.exp(1j*n*phi[rho > a])
            )

            et[rho <= a] = et[rho <= a] + (
                E0*1j**(-n)*cn[i]*jv(n, kd*rho[rho <= a])
                * np.exp(1j*n*phi[rho <= a])
            )

            i += 1

    else:
        S = theta.size
        et = np.zeros((x.size, S), dtype=complex)
        for s in range(S):
            xp, yp = rotate_axis(theta[s], x.reshape(-1), y.reshape(-1))
            rho, phi = cart2pol(xp, yp)
            i = 0
            for n in N:

                et[rho > a, s] = et[rho > a, s] + (
                    E0*1j**(-n)*(jv(n, kb*rho[rho > a])
                                 + an[i]*hankel2(n, kb*rho[rho > a]))
                    * np.exp(1j*n*phi[rho > a])
                )

                et[rho <= a, s] = et[rho <= a, s] + (
                    E0*1j**(-n)*cn[i]*jv(n, kd*rho[rho <= a])
                    * np.exp(1j*n*phi[rho <= a])
                )

                i += 1
    return et

def rotate_axis(theta, x, y):
    """Summarize the method."""
    T = np.array([[np.cos(theta), np.sin(theta)],
                  [-np.sin(theta), np.cos(theta)]])
    r = np.vstack((x.reshape(-1), y.reshape(-1)))
    rp = T@r
    xp, yp = np.vsplit(rp, 2)
    xp = np.reshape(np.squeeze(xp), x.shape)
    yp = np.reshape(np.squeeze(yp), y.shape)
    return xp, yp

def cart2pol(x, y):
    """Summarize the method."""
    rho = np.sqrt(x**2+y**2)
    phi = np.arctan2(y, x)
    phi[phi < 0] = 2*pi + phi[phi < 0]
    return rho, phi

def compute_scattered_field(xm, ym, an, n, kb, theta, magnitude):
    """Summarize the method."""
    M, S, N = xm.size, theta.size, round((an.size-1)/2)
    E0 = magnitude
    es = np.zeros((M, S), dtype=complex)
    for s in range(S):
        xp, yp = rotate_axis(theta[s], xm, ym)
        rho, phi = cart2pol(xp, yp)
        for j in range(phi.size):
            es[j, s] = E0*np.sum(1j**(-n)*an*hankel2(n, kb*rho[j])
                                 * np.exp(1j*n*phi[j]))

    return es
